# **Tạo file index JSON cho các Span Adaptation Vectors (Inner Content) đã được sinh bởi script `create_span_vectors_inner_content.py`.**

## Mục đích

- Tự động quét các thư mục chứa span adaptation vectors (các file `.pt` được lưu cho từng câu và từng argument).
- Trích xuất thông tin cấu trúc từ tên file:
  - Chỉ số câu: `sentence_idx`.
  - Chỉ số argument: `ARG-x`.
- Gom nhóm các thông tin này theo từng động từ (`verb_name`) và lưu thành các file index `.json`.
- Hỗ trợ việc tra cứu, nạp và xử lý span vectors sau này mà không cần duyệt và parse lại tên file thủ công."

## Đầu vào

1. Cấu hình đường dẫn cơ bản
   - `drive_base_path`:
     - Đường dẫn gốc trên Google Drive:
       - `/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep`.

2. Cấu hình các bộ dữ liệu cần tạo index: `datasets_to_index`
   - Danh sách các tuple:
     - `(dataset_name, input_dir, output_dir)`.
   - Trong đó:
     - `dataset_name`: tên bộ dữ liệu (ví dụ: `"Train_GramVar"`, `"Test_ParaVE"`).
     - `input_dir`: thư mục chứa các file `.pt` span vectors (được tạo trước bởi script span vectors).
     - `output_dir`: thư mục sẽ dùng để lưu các file index `.json`.

   - Ví dụ:
     - `span_adaptation_vectors_train_gramvar_inner_content` → `verb_indexes_span_train_gramvar_inner_content`.
     - `span_adaptation_vectors_test_parave_inner_content` → `verb_indexes_span_test_parave_inner_content`.

3. Cấu trúc thư mục input
   - Trong `input_dir`, các span vectors được tổ chức như sau:
     - Mỗi động từ là một thư mục con: `<verb_name>/`.
     - Bên trong mỗi `<verb_name>/` là các file:
       - `sentence_<i>_ARG_<k>.pt`
         - `<i>`: chỉ số câu (sentence index).
         - `<k>`: chỉ số argument (ví dụ: `0`, `1`, ...).

4. Định dạng tên file
   - Script sử dụng regex:
     - `file_pattern = re.compile(r"sentence_(\d+)_ARG_(\d+)\.pt")`
   - Chỉ các file khớp đúng định dạng này mới được đưa vào index.

## Đầu ra

1. Các file index JSON theo từng động từ
   - Với mỗi thư mục động từ `<verb_name>` trong `input_dir`, nếu có file hợp lệ, script sẽ tạo:
     - Một file JSON tại:
       - `output_dir/<verb_name>.json`.

2. Cấu trúc nội dung file JSON
   - Mỗi file JSON chứa một list các object có cấu trúc:
     - `verb_name`: tên động từ (tương ứng thư mục đang xử lý).
     - `sentence_idx`: chỉ số câu, lấy từ phần `sentence_<i>` trong tên file.
     - `arg_label`: nhãn dạng `"ARG-x"` (x được lấy từ phần `_ARG_<x>` trong tên file).
     - `vector_file_path`: đường dẫn đầy đủ tới file `.pt`.

   - Ví dụ một phần tử:
     ```json
     {
         "verb_name": "bind",
         "sentence_idx": 12,
         "arg_label": "ARG-0",
         "vector_file_path": "/.../span_adaptation_vectors_train_gramvar_inner_content/bind/sentence_12_ARG_0.pt"
     }
     ```


## Quy trình

### 1. Thiết lập và cấu hình

- Import các thư viện:
  - `torch`, `os`, `glob`, `json`, `tqdm.auto.tqdm`, `re`.
- Khai báo `drive_base_path`.
- Khai báo `datasets_to_index`:
  - Gắn `input_dir` và `output_dir` cho bốn nhóm:
    - Train_GramVar, Train_ParaVE, Test_GramVar, Test_ParaVE.

### 2. Định nghĩa regex để parse tên file

- `file_pattern = re.compile(r"sentence_(\d+)_ARG_(\d+)\.pt")`.
- Mục đích:
  - Trích xuất:
    - `(\d+)` thứ nhất: `sentence_idx`.
    - `(\d+)` thứ hai: `arg_num` (để dựng lại `ARG-<arg_num>`).

### 3. Hàm `create_indexes_for_spans(input_dir, output_dir, dataset_name)`

- Chức năng:
  - Quét một `input_dir` cụ thể.
  - Với từng động từ (mỗi thư mục con), tạo file index JSON chứa thông tin các span vectors.

- Các bước chi tiết:

1. In thông tin về dataset đang xử lý:
   - Tên dataset, thư mục input, thư mục output.

2. Tạo `output_dir` nếu chưa tồn tại:
   - `os.makedirs(output_dir, exist_ok=True)`.

3. Tìm tất cả thư mục động từ trong `input_dir`:
   - `verb_dirs = [d for d in glob.glob(os.path.join(input_dir, '*')) if os.path.isdir(d)]`.
   - Nếu không có `verb_dirs`:
     - In cảnh báo và `return`.

4. Khởi tạo `tqdm` để theo dõi tiến độ theo từng động từ:
   - `pbar = tqdm(total=len(verb_dirs), desc=f"Xử lý {dataset_name}", unit="verb")`.

5. Với mỗi `verb_dir` trong `verb_dirs`:
   - Xác định `verb_name = os.path.basename(verb_dir)`.
   - Lấy list file `.pt` trong thư mục này:
     - `span_vector_files = glob.glob(os.path.join(verb_dir, '*.pt'))`.
   - Khởi tạo `index_data_for_verb = []`.

6. Duyệt qua từng file `.pt`:
   - Lấy `filename = os.path.basename(file_path)`.
   - Dùng regex:
     - `match = file_pattern.match(filename)`.
   - Nếu tên file khớp pattern:
     - `sentence_idx = int(match.group(1))`.
     - `arg_num = match.group(2)`.
     - `arg_label = f"ARG-{arg_num}"`.
     - Thêm vào `index_data_for_verb` một dict:
       - `verb_name`, `sentence_idx`, `arg_label`, `vector_file_path`.
   - Nếu không khớp:
     - In cảnh báo qua `pbar.write` và bỏ qua file đó.

7. Sau khi xử lý xong tất cả file của một động từ:
   - Nếu `index_data_for_verb` không rỗng:
     - Tạo đường dẫn JSON output:
       - `output_json_path = os.path.join(output_dir, f"{verb_name}.json")`.
     - Ghi file JSON:
       - `json.dump(index_data_for_verb, f, indent=4, ensure_ascii=False)`.

8. Cập nhật tiến độ:
   - `pbar.update(1)` sau mỗi `verb_dir`.

9. Đóng `pbar` và in thông báo số lượng động từ đã tạo index.

### 4. Hàm main

- Duyệt qua `datasets_to_index`:
  - Với mỗi `(name, input_dir, output_dir)`:
    - Gọi: `create_indexes_for_spans(input_dir, output_dir, name)`.

In [ ]:
# Import các thư viện cần thiết
import torch
import os
import glob
import json
from tqdm.auto import tqdm
import re # Thêm regex

# THIẾT LẬP CÁC ĐƯỜNG DẪN
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# *** CẤU HÌNH QUÉT ***
# Script này sẽ quét các thư mục output của file 'create_span_vectors_inner_content.py'
# và tạo ra các file index .json cho chúng.

datasets_to_index = [
    # --- Dữ liệu Train ---
    (
        "Train_GramVar",
        # Thư mục INPUT (nơi chứa các file .pt của span vector)
        os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Train/span_adaptation_vectors_train_gramvar_inner_content'),
        # Thư mục OUTPUT (nơi lưu file index .json)
        os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Train_Index/verb_indexes_span_train_gramvar_inner_content')
    ),
    (
        "Train_ParaVE",
        os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Train/span_adaptation_vectors_train_parave_inner_content'),
        os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Train_Index/verb_indexes_span_train_parave_inner_content')
    ),
    # --- Dữ liệu Test ---
    (
        "Test_GramVar",
        os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Test/span_adaptation_vectors_test_gramvar_inner_content'),
        os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Test_Index/verb_indexes_span_test_gramvar_inner_content')
    ),
    (
        "Test_ParaVE",
        os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Test/span_adaptation_vectors_test_parave_inner_content'),
        os.path.join(drive_base_path, 'Span Adaptation Vector/With Weight/Test_Index/verb_indexes_span_test_parave_inner_content')
    ),
]

# CÁC HÀM XỬ LÝ

# Regex để trích xuất thông tin từ tên file
# Ví dụ: "sentence_123_ARG_0.pt"
file_pattern = re.compile(r"sentence_(\d+)_ARG_(\d+)\.pt")

def create_indexes_for_spans(input_dir, output_dir, dataset_name):
    """
    Quét thư mục input (đã tạo bởi create_span_vectors...),
    và tạo ra các file index .json (gom nhóm theo động từ) trong thư mục output.
    """
    print(f"\n=================================================")
    print(f"BẮT ĐẦU TẠO INDEX CHO: {dataset_name}")
    print(f" -> Đang quét: {input_dir}")
    print(f" -> Sẽ lưu vào: {output_dir}")
    print(f"=================================================")

    os.makedirs(output_dir, exist_ok=True)

    # Tìm tất cả các thư mục động từ trong thư mục input
    verb_dirs = [d for d in glob.glob(os.path.join(input_dir, '*')) if os.path.isdir(d)]

    if not verb_dirs:
        print(f"CẢNH BÁO: Không tìm thấy thư mục động từ nào trong '{input_dir}'. Bỏ qua.")
        return

    pbar = tqdm(total=len(verb_dirs), desc=f"Xử lý {dataset_name}", unit="verb")

    for verb_dir in verb_dirs:
        verb_name = os.path.basename(verb_dir)
        span_vector_files = glob.glob(os.path.join(verb_dir, '*.pt'))

        index_data_for_verb = []

        for file_path in span_vector_files:
            filename = os.path.basename(file_path)
            match = file_pattern.match(filename)

            if match:
                sentence_idx = int(match.group(1))
                arg_num = match.group(2)
                arg_label = f"ARG-{arg_num}" # Tạo lại ARG-0, ARG-1

                index_data_for_verb.append({
                    'verb_name': verb_name,
                    'sentence_idx': sentence_idx,
                    'arg_label': arg_label,
                    'vector_file_path': file_path
                })
            else:
                pbar.write(f"CẢNH BÁO: Tên file không khớp {filename}. Bỏ qua.")

        # Nếu có dữ liệu index cho động từ này, lưu lại
        if index_data_for_verb:
            output_json_path = os.path.join(output_dir, f"{verb_name}.json")
            with open(output_json_path, 'w', encoding='utf-8') as f:
                json.dump(index_data_for_verb, f, indent=4, ensure_ascii=False)

        pbar.update(1)

    pbar.close()
    print(f" -> Đã tạo index thành công cho {len(verb_dirs)} động từ.")

# THỰC THI CHƯƠNG TRÌNH
if __name__ == "__main__":
    print("--- Bắt đầu quá trình tạo file Index cho Span Vectors (v2 - Inner Content) ---")

    for name, input_dir, output_dir in datasets_to_index:
        create_indexes_for_spans(input_dir, output_dir, name)

    print("\n--- TẠO INDEX HOÀN TẤT! ---")

--- Bắt đầu quá trình tạo file Index cho Span Vectors (v2 - Inner Content) ---

BẮT ĐẦU TẠO INDEX CHO: Train_GramVar
 -> Đang quét: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_vectors_train_gramvar_inner_content
 -> Sẽ lưu vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_span_train_gramvar_inner_content


Xử lý Train_GramVar:   0%|          | 0/35 [00:00<?, ?verb/s]

 -> Đã tạo index thành công cho 35 động từ.

BẮT ĐẦU TẠO INDEX CHO: Train_ParaVE
 -> Đang quét: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_vectors_train_parave_inner_content
 -> Sẽ lưu vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_span_train_parave_inner_content


Xử lý Train_ParaVE:   0%|          | 0/35 [00:00<?, ?verb/s]

 -> Đã tạo index thành công cho 35 động từ.

BẮT ĐẦU TẠO INDEX CHO: Test_GramVar
 -> Đang quét: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_vectors_test_gramvar_inner_content
 -> Sẽ lưu vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_span_test_gramvar_inner_content


Xử lý Test_GramVar:   0%|          | 0/35 [00:00<?, ?verb/s]

 -> Đã tạo index thành công cho 35 động từ.

BẮT ĐẦU TẠO INDEX CHO: Test_ParaVE
 -> Đang quét: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/span_adaptation_vectors_test_parave_inner_content
 -> Sẽ lưu vào: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/verb_indexes_span_test_parave_inner_content


Xử lý Test_ParaVE:   0%|          | 0/35 [00:00<?, ?verb/s]

 -> Đã tạo index thành công cho 35 động từ.

--- TẠO INDEX HOÀN TẤT! ---
